# This will be the Notebook that we use to analyze whether there has been a rise in patents surrounding online meeting technology patents after the start of the Covid Pandemic

In [1]:
# Here, we are going to load the generic g_patent and g_cpc_current DataFrames as well as our pandas, matplotlib etc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set( style = 'white' )

In [4]:
df_patent = pd.read_csv("g_patent.tsv", delimiter='\t', dtype={'patent_id': str, 
                                                        'patent_type': str, 
                                                        'patent_title': str,
                                                        'patent_abstract': str,
                                                        'wipo_kind': str,
                                                        'num_claims': int,
                                                        'withdrawn': int,
                                                        'filename': str}, parse_dates=[2])
df_patent.head(20)

,patent_id,patent_type,patent_date,patent_title,patent_abstract,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,A frequency modulated (coherent) laser detecti...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,The injection molding machine includes a fixed...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,The present invention relates to: a method for...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,The invention relates to a method for producin...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",The present invention relates to provides a do...,B2,6,0,ipg180619.xml
5,10000005,utility,2018-06-19,Article vacuum formation method and vacuum for...,A vacuum forming apparatus is provided that fo...,B2,4,0,ipg180619.xml
6,10000006,utility,2018-06-19,Thermoforming mold device and a process for it...,A thermoforming mold device (1) providing a pi...,B2,8,0,ipg180619.xml
7,10000007,utility,2018-06-19,PEX expanding tool,An expanding tool comprising: an actuator comp...,B2,24,0,ipg180619.xml
8,10000008,utility,2018-06-19,Bracelet mold and method of use,"A decorated strip of coated, heat-shrinkable, ...",B2,11,0,ipg180619.xml
9,10000009,utility,2018-06-19,Sterile environment for additive manufacturing,"In sterile, additive manufacturing wherein one...",B2,21,0,ipg180619.xml


In [5]:
df_cpc = pd.read_table("g_cpc_current.tsv", delimiter="\t", dtype={"patent_id": int,
                                                               "cpc_sequence": int,
                                                               "cpc_section": str,
                                                               "cpc_subclass": str,
                                                               "cpc_group": str,
                                                               "cpc_type": str,
                                                               "cpc_symbol_position": str})
df_cpc.head(20)

,patent_id,cpc_sequence,cpc_section,cpc_class,cpc_subclass,cpc_group,cpc_type,cpc_symbol_position
0,4796895,1,F,F16,F16H,F16H61/00,inventional,NaN
1,10913199,0,B,B29,B29C,B29C55/08,inventional,NaN
2,5208443,0,B,B29,B29C,B29C65/366,inventional,NaN
3,7830588,6,G,G09,G09G,G09G2310/0275,additional,NaN
4,7232943,1,A,A01,A01H,A01H5/10,inventional,NaN
5,10815370,2,C,C08,C08F,C08F265/08,inventional,NaN
6,8271025,4,H,H04,H04M,H04M15/00,inventional,NaN
7,8208778,1,G,G02,G02B,G02B6/12002,inventional,NaN
8,10299603,18,B,B64,B64D,B64D11/00154,inventional,NaN
9,10941581,24,B,B32,B32B,B32B2255/10,additional,NaN


In [15]:
# Here, we are going to filter our g_patent and g_cpc_current datasets so that we only get the patents
# that are related to technology, and that are 2020 and above
tec_df = df_cpc.loc[df_cpc['cpc_section'] == "Y"]
tech_df = tech_df.drop(columns=["cpc_sequence", "cpc_class", "cpc_subclass", "cpc_group", "cpc_symbol_position", "cpc_type"], axis=1)
tech_df['patent_id'] = tech_df['patent_id'].astype(str)
tech_df = pd.merge(df_patent.drop(axis=1, columns=df_patent.columns[5:]), tech_df, on="patent_id")
tech_df = tech_df[(tech_df['patent_date'] > "2019-12-31")]
tech_df['full_patent_details'] = tech_df[['patent_title', 'patent_abstract']].agg(' '.join, axis=1)
tech_df.drop(columns=["patent_title", "patent_date"], axis=1, inplace=True)

In [16]:
tech_df

,patent_id,patent_type,patent_abstract,cpc_section,full_patent_details
116108,10524409,utility,"In one embodiment, one or more tangible, non-t...",Y,System and method for controlling agricultural...
116109,10524425,utility,One variation of a method for automating trans...,Y,Method for automating transfer of plants withi...
116110,10524426,utility,A method of stimulating plant growth in a cont...,Y,Light sources adapted to spectral sensitivity ...
116111,10524430,utility,An irrigation management system monitors irrig...,Y,Irrigation management system An irrigation man...
116112,10524432,utility,System and methods for monitoring the growth o...,Y,Systems and methods for cultivating and distri...
...,...,...,...,...,...
287756,11457532,utility,A method for manufacturing a circuit board com...,Y,Method of manufacturing circuit board A method...
287757,11457535,utility,A metallic housing of an electronic device inc...,Y,Metallic housing of electronic device A metall...
287758,11457536,utility,An electronic device includes a flexible displ...,Y,"Protective film, electronic device having same..."
287759,11457547,utility,A thermal management system includes an auxili...,Y,Phase change thermal management system for ele...


In [19]:
meetings_df = tech_df.loc[tech_df['full_patent_details'].str.contains('meeting', case=False)]
meetings_df

,patent_id,patent_type,patent_abstract,cpc_section,full_patent_details
116692,10526934,utility,A friction loss management system for an engin...,Y,Cylinder deactivation control and methods A fr...
117043,10529332,utility,"At an electronic device with a display, a micr...",Y,Virtual assistant activation At an electronic ...
119961,10543653,utility,A container assembly for use in a high-pressur...,Y,Construction and composition of preformed cont...
124202,10563549,utility,A friction loss management system for an engin...,Y,Cylinder deactivation control and methods A fr...
125499,10570652,utility,A sash fastener for securing a slidable and ti...,Y,Integrated sash lock and tilt latch combinatio...
...,...,...,...,...,...
283177,11440842,utility,The invention belongs to the technical field o...,Y,Method for preparing a cementing material usin...
283178,11440842,utility,The invention belongs to the technical field o...,Y,Method for preparing a cementing material usin...
283179,11440842,utility,The invention belongs to the technical field o...,Y,Method for preparing a cementing material usin...
283251,11441086,utility,"For the shipping industry, these fuels provide...",Y,Environment-friendly marine fuel For the shipp...
